# Spectrophotometric Analysis
This notebooks is written to analyze data from a colorimetric techniqes. The data must be in the proper format to read by this script.

In [ ]:
import numpy as np
import pandas as pd
from IPython import display
from scipy.optimize import curve_fit

# What experiment does the data belong to and how many samples do you have?


In [ ]:
# Indicate the name of the experiment (e.g. "HFO" or "Bacteria1")
exp_name = "HFO"

# Indicate how many samples you have
n_samples = 11

# creation of a dataframe to hold your data plus one additional line for experimental parameters
df = pd.DataFrame(
    index=np.arange(n_samples + 1),
    columns=[
        "std",
        "abs",
        "sample_name",
        "pH",
        "sampabs",
        "tot_zn_ppm",
    ],
)

# Data input
Replace the data for `standards`, `stdabs`, `sample_name`, `pH`, `sampleabs`, `tot_zn_ppm` with your own data. Other parameters are already filled in for you.

In [ ]:
# Zinc concentrations of the standards (already correct)
standards = [0.46, 1.03, 3.02, 6.04]
# Replace the data with the absorbance of your standards, lowest concentration to highest
stdabs = [0.023, 0.052, 0.182, 0.292]
# Replace the data with your sample names, be sure the names are within quotes
sample_name = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
# Replace the data with your sample pH values
pH = [3.25, 4.74, 5.61, 4.96, 5.57, 5.73, 6.19, 6.46, 7.16, 7.66, 7.8, 4.73]
# Replace the data with the absorbance values for your samples
sampabs = [
    0.224,
    0.383,
    0.171,
    0.164,
    0.115,
    0.105,
    0.101,
    0.083,
    -0.007,
    0.054,
    0.033,
    0.043,
]
# Replace the data with the total zinc concentration of your experiment
tot_zn_ppm = 5

# Check input data
Run the next cell and make sure it looks like the example data above

In [ ]:
df.loc[0:3, "std"] = standards
df.loc[0:3, "abs"] = stdabs
df.loc[0 : len(sample_name) - 1, "sample_name"] = sample_name
df.loc[0 : len(sample_name) - 1, "pH"] = pH
df.loc[0 : len(sample_name) - 1, "sampabs"] = sampabs
df.loc[0, "tot_zn_ppm"] = tot_zn_ppm
df

# Calibration curve
Now that the data is uploaded we need to perform a linear regression to create the calibration curve. We will use the Beer-Lambert law (or Beer's law), where $A$ is absorbance, $\epsilon$ is the molar absorptivity, $l$ is path length, and $C$ is concentration, to create our calibration curve.  

$$A=\epsilon{l}C$$

We can see that Beer's law is the equation for a straight line with an intercept of zero. Theoretically this makes sense in that if there are no absorbing molecules present then the absorbance should be zero. Realistically this doesn't happen. For example, analytical instruments don't behave ideally (i.e., there is always some background noise in signals) and blank substractions aren't perfect. So our best fit line will include an intercept. A calibration shouldn't go through the origin unless it demonstrated that the intercept is not statistically different than zero by calculating a t-ratio. For our spectrophotometric analysis the intercept will be statistically different than zero so we must include it.

There are many different ways to fit linear models in Python. I've chosen to use the SciPy module just demonstrate a different way of fitting than is found in the ferrozine analysis script.

In [ ]:
# Use SciPy to determine our calibration curve
# define a function for the equation of a straight line
def func(x, p1, p2):
    return x * p1 + p2


# rename the calibration data x and y and select range with data leaving out the nans
x = df["std"][0:4]
y = df["abs"][0:4]
# run the regression and save the output parameter (slope and intercept) in popt and save the covariance matrix in pcov
popt, pcov = curve_fit(func, x, y, p0=[0.003, 0])
# calculate the coefficient of determination also known as r^2
residuals = y - func(x, popt[0], popt[1])
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# intercept
b = popt[1]
# slope
m = popt[0]
# print fit parameters and error
print("Calibration equation\t r-squared")
print("y = %.4f*x + %.4f\t %f" % (m, b, r_squared))
print("\n")
# inverse prediction of concentration from absorbance measurements
df["sampconc"] = (df["sampabs"] - b) / m
print(df)

# Save the data to a csv file.


In [ ]:
# To save the dataframe to a CSV file with the date as the filename it is necessary 
# to import the datetime module
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Format the date as a string
date_string = now.strftime("%Y-%m-%d")

# Save the dataframe to a CSV file with the date as the filename
filename = f"{exp_name+'_'+date_string}.csv"
df.to_csv(filename, index=False)
print('The data is saved in file: %s' % filename)

# Copy data to class spreadsheet
### Open the saved data file by double clicking the file name in the directory to the right. Copy the `pH` and `sampconc` columns to the spreadsheet: [data spreadsheet](https://uwnetid-my.sharepoint.com/:x:/g/personal/dgormanl_uw_edu/Ea6sn5NjzvFJnkofm11_F2UBwFd0ZspGA36oza-ttW7JFA?e=cekeZZ)